
### Търсене и извличане на информация. Приложение на дълбоко машинно обучение
### Стоян Михов
### Зимен семестър 2025/2026

## Упражнение 10

За да работи програмата трябва корпуса от публицистични текстове за Югоизточна Европа,
да се намира разархивиран в наддиректорията, в която е програмата (виж упражнение 2).

Преди да се стартира програмата е необходимо да се активира съответното обкръжение с командата:
conda activate tii

In [1]:
import sys
import nltk
from nltk.corpus import PlaintextCorpusReader
import numpy as np
import torch
import random
import math

In [ ]:
import torch
print(list(torch.__path__))

['d:\\Anaconda3\\envs\\tii\\Lib\\site-packages\\torch']


In [2]:
X = torch.tensor([[1.2,2,3],[4,5,6]])
Y = torch.tensor([[3,2,1],[2,3,4.1]], requires_grad=True)
A = torch.rand(3, requires_grad=True)
B = X+Y
C = torch.matmul(B,A)
D = -2 * C
E = torch.sum(D)

In [3]:
B.retain_grad()
C.retain_grad()
D.retain_grad()
E.retain_grad()

In [4]:
E.backward()

In [10]:
print('X=',X)
print('Y=',Y)
print('A=',A)
print('B=',B)
print('C=',C)
print('D=',D)
print('E=',E)

print('grad E =',E.grad)
print('grad D =',D.grad)
print('grad C =',C.grad)
print('grad B =',B.grad)

print('grad A =',A.grad)
print('grad Y =',Y.grad)

X= tensor([[1.2000, 2.0000, 3.0000],
        [4.0000, 5.0000, 6.0000]])
Y= tensor([[2.4404, 1.6861, 0.9917],
        [1.4404, 2.6861, 4.0917]], requires_grad=True)
A= tensor([-2.0673, -2.4768, -3.0462], requires_grad=True)
B= tensor([[ 4.2000,  4.0000,  4.0000],
        [ 6.0000,  8.0000, 10.1000]], grad_fn=<AddBackward0>)
C= tensor([6.0654, 9.9210], grad_fn=<MvBackward0>)
D= tensor([-12.1307, -19.8421], grad_fn=<MulBackward0>)
E= tensor(-31.9728, grad_fn=<SumBackward0>)
grad E = tensor(1.)
grad D = tensor([1., 1.])
grad C = tensor([-2., -2.])
grad B = tensor([[-1.8655, -1.0464, -0.0275],
        [-1.8655, -1.0464, -0.0275]])
grad A = tensor([2.5000, 2.5000, 2.5500])
grad Y = tensor([[0.4664, 0.2616, 0.0069],
        [0.4664, 0.2616, 0.0069]])


In [6]:
S = torch.mean(A)
S.backward()
print('grad A =',A.grad)
print('grad Y =',Y.grad)

grad A = tensor([-20.0667, -23.6667, -27.8667])
grad Y = tensor([[-1.8655, -1.0464, -0.0275],
        [-1.8655, -1.0464, -0.0275]])


In [7]:
A.grad = None
Y.grad = None

S = torch.mean(torch.matmul(Y,A))
S.backward()

In [8]:
print('grad A =',A.grad)
print('grad Y =',Y.grad)

with torch.no_grad():
    A -= 1.2 * A.grad
    Y -= 1.2 * Y.grad

print('Y=',Y)

grad A = tensor([2.5000, 2.5000, 2.5500])
grad Y = tensor([[0.4664, 0.2616, 0.0069],
        [0.4664, 0.2616, 0.0069]])
Y= tensor([[2.4404, 1.6861, 0.9917],
        [1.4404, 2.6861, 4.0917]], requires_grad=True)


In [ ]:
class progressBar:
    def __init__(self ,barWidth = 50):
        self.barWidth = barWidth
        self.period = None
    def start(self, count):
        self.item=0
        self.period = int(count / self.barWidth)
        sys.stdout.write("["+(" " * self.barWidth)+"]")
        sys.stdout.flush()
    def tick(self):
        if self.item>0 and self.item % self.period == 0:
            progress = self.item // self.period
            sys.stdout.write(f"\r[{'-'* progress}{' '*(self.barWidth - progress)}]")
            sys.stdout.flush()
        self.item += 1
    def stop(self):
        sys.stdout.write("\r["+("-" * self.barWidth)+"]\n")

def extractDictionary(corpus, limit=20000):
    pb = progressBar()
    pb.start(len(corpus))
    dictionary = {}
    for doc in corpus:
        pb.tick()
        for w in doc:
            if w not in dictionary: dictionary[w] = 0
        dictionary[w] += 1
    L = sorted([(w,dictionary[w]) for w in dictionary], key = lambda x: x[1] , reverse=True)
    if limit > len(L): limit = len(L)
    words = [ w for w,_ in L[:limit] ] + [unkToken]
    word2ind = { w:i for i,w in enumerate(words)}
    pb.stop()
    return words, word2ind

def splitSentCorpus(fullSentCorpus, testFraction = 0.1):
    random.seed(42)
    random.shuffle(fullSentCorpus)
    testCount = int(len(fullSentCorpus) * testFraction)
    testSentCorpus = fullSentCorpus[:testCount]
    trainSentCorpus = fullSentCorpus[testCount:]
    return testSentCorpus, trainSentCorpus

def extractData(corpus, order, word2ind):
    pb = progressBar()
    pb.start(len(corpus))
    unk = word2ind[unkToken]
    start = word2ind[startToken]

    points = sum(len(s)-1 for s in corpus)
    
    target = np.empty(points, dtype='int32')
    context = np.empty((points,order-1), dtype='int32')
    p = 0
    for doc in corpus:
        pb.tick()
        for wi in range(1,len(doc)):
            i = word2ind.get(doc[wi], unk)
            target[p] = i
            for k in range(1,order):
                if wi-k < 0:
                    j = start
                else:
                    j = word2ind.get(doc[wi-k], unk)
                context[p,k-1] = j
            p += 1
    pb.stop()
    return target, context

In [12]:
corpus_root = '../JOURNALISM.BG/C-MassMedia'
myCorpus = PlaintextCorpusReader(corpus_root, '.*\.txt')
startToken = '<START>'
endToken = '<END>'
unkToken = '<UNK>'

corpus = [ [startToken] + [w.lower() for w in sent] + [endToken] for sent in myCorpus.sents()]

words, word2ind = extractDictionary(corpus)
testCorpus, trainCorpus  = splitSentCorpus(corpus, testFraction = 0.01)

order = 4
target, context = extractData(trainCorpus, order, word2ind)
testtarget, testcontext = extractData(testCorpus, order, word2ind)

emb_size = 50
hid_size = 100

L = len(words)

batchSize = 1000
idx = np.arange(len(target), dtype='int32')
np.random.shuffle(idx)
learning_rate = 1.

[                                                  --------------------------------------------------]
[                                                  --------------------------------------------------]
[                                                  --------------------------------------------------]


In [13]:
E = torch.empty(L, emb_size, requires_grad = True)
W1 = torch.empty((order-1)*emb_size, hid_size, requires_grad = True)
b1 = torch.empty(hid_size, requires_grad = True)
W2 = torch.empty(hid_size, L, requires_grad = True)
b2 = torch.empty(L, requires_grad = True)

torch.nn.init.normal_(E)
torch.nn.init.normal_(W1)
torch.nn.init.normal_(b1)
torch.nn.init.normal_(W2)
torch.nn.init.normal_(b2)

sigmoid_fn = torch.nn.Sigmoid()

for b in range(0,len(idx),batchSize):

    batchIdx = idx[b:min(b+batchSize,len(idx))]
    S = len(batchIdx)
    batchTarget = torch.tensor(target[batchIdx], dtype=torch.long)
    batchContext = context[batchIdx]

    X = E[batchContext].view(S,(order-1) * emb_size)
    h = sigmoid_fn(torch.matmul(X,W1) + b1.unsqueeze(0))
    z = torch.matmul(h,W2) + b2.unsqueeze(0)
    
    t = torch.exp(z)
    s = torch.sum(t,axis=1)
    u = t/s.unsqueeze(1)
    p = u[torch.arange(S),batchTarget]
    H = -torch.mean(torch.log(p))

    H.backward()

    with torch.no_grad():
        E -= learning_rate * E.grad
        W1 -= learning_rate * W1.grad
        b1 -= learning_rate * b1.grad
        W2 -= learning_rate * W2.grad
        b2 -= learning_rate * b2.grad
        # Manually zero the gradients
        E.grad = None
        W1.grad = None
        b1.grad = None
        W2.grad = None
        b2.grad = None

    if b % 10000 == 0:
        print(b, '/', len(idx), H.item())

0 / 9892431 28.993371963500977
10000 / 9892431 21.868349075317383
20000 / 9892431 19.250085830688477
30000 / 9892431 17.291868209838867
40000 / 9892431 16.418853759765625
50000 / 9892431 15.887689590454102
60000 / 9892431 15.69060230255127
70000 / 9892431 15.147296905517578
80000 / 9892431 15.223397254943848
90000 / 9892431 14.322261810302734
100000 / 9892431 13.67988109588623
110000 / 9892431 13.562220573425293
120000 / 9892431 13.311437606811523
130000 / 9892431 12.684711456298828
140000 / 9892431 12.878501892089844
150000 / 9892431 12.44346809387207
160000 / 9892431 12.121983528137207
170000 / 9892431 12.398822784423828
180000 / 9892431 12.197887420654297
190000 / 9892431 12.110041618347168
200000 / 9892431 11.38534164428711
210000 / 9892431 11.151517868041992
220000 / 9892431 11.232037544250488
230000 / 9892431 10.916915893554688
240000 / 9892431 10.827771186828613
250000 / 9892431 10.245821952819824
260000 / 9892431 10.078028678894043
270000 / 9892431 10.27798843383789
280000 / 98

In [14]:
torch.nn.init.normal_(E)
torch.nn.init.normal_(W1)
torch.nn.init.normal_(b1)
torch.nn.init.normal_(W2)
torch.nn.init.normal_(b2)

loss_fn = torch.nn.CrossEntropyLoss()

for b in range(0,len(idx),batchSize):
    
    batchIdx = idx[b:min(b+batchSize,len(idx))]
    S = len(batchIdx)
    batchTarget = torch.tensor(target[batchIdx], dtype=torch.long)
    batchContext = context[batchIdx]
    
    X = E[batchContext].view(S,(order-1) * emb_size)
    h = sigmoid_fn(torch.matmul(X,W1) + b1)
    z = torch.matmul(h,W2) + b2
    H = loss_fn(z,batchTarget)
    
    H.backward()
    
    with torch.no_grad():
        E -= learning_rate * E.grad
        W1 -= learning_rate * W1.grad
        b1 -= learning_rate * b1.grad
        W2 -= learning_rate * W2.grad
        b2 -= learning_rate * b2.grad
        # Manually zero the gradients
        E.grad = None
        W1.grad = None
        b1.grad = None
        W2.grad = None
        b2.grad = None
    
    if b % 10000 == 0:
        print(b, '/', len(idx), H.item())

0 / 9892431 28.47962760925293
10000 / 9892431 20.554868698120117
20000 / 9892431 18.138824462890625
30000 / 9892431 16.304046630859375
40000 / 9892431 15.938725471496582
50000 / 9892431 15.586565971374512
60000 / 9892431 15.152789115905762
70000 / 9892431 14.478547096252441
80000 / 9892431 13.899019241333008
90000 / 9892431 13.75214672088623
100000 / 9892431 12.956308364868164
110000 / 9892431 12.644067764282227
120000 / 9892431 12.358532905578613
130000 / 9892431 11.945523262023926
140000 / 9892431 11.966723442077637
150000 / 9892431 11.804121017456055
160000 / 9892431 11.135963439941406
170000 / 9892431 11.548476219177246
180000 / 9892431 11.19436264038086
190000 / 9892431 11.190728187561035
200000 / 9892431 10.674700736999512
210000 / 9892431 10.45792293548584
220000 / 9892431 10.226217269897461
230000 / 9892431 10.216938018798828
240000 / 9892431 10.10419750213623
250000 / 9892431 9.77971363067627
260000 / 9892431 9.45079517364502
270000 / 9892431 9.889200210571289
280000 / 9892431

In [15]:

device = torch.device("cuda:0")
device = torch.device("cpu")

E = torch.empty(L, emb_size, requires_grad = True, device = device)
W1 = torch.empty((order-1)*emb_size, hid_size, requires_grad = True, device = device)
b1 = torch.empty(hid_size, requires_grad = True, device = device)
W2 = torch.empty(hid_size, L, requires_grad = True, device = device)
b2 = torch.empty(L, requires_grad = True, device = device)

torch.nn.init.normal_(E)
torch.nn.init.normal_(W1)
torch.nn.init.normal_(b1)
torch.nn.init.normal_(W2)
torch.nn.init.normal_(b2)

np.random.shuffle(idx)
for b in range(0,len(idx),batchSize):
    
    batchIdx = idx[b:min(b+batchSize,len(idx))]
    S = len(batchIdx)
    batchTarget = torch.tensor(target[batchIdx], dtype=torch.long, device = device)
    batchContext = context[batchIdx]
    
    X = E[batchContext].view(S,(order-1) * emb_size)
    h = sigmoid_fn(torch.matmul(X,W1) + b1)
    z = torch.matmul(h,W2) + b2
    H = loss_fn(z,batchTarget)
    
    H.backward()
    
    with torch.no_grad():
        E -= learning_rate * E.grad
        W1 -= learning_rate * W1.grad
        b1 -= learning_rate * b1.grad
        W2 -= learning_rate * W2.grad
        b2 -= learning_rate * b2.grad
        # Manually zero the gradients
        E.grad = None
        W1.grad = None
        b1.grad = None
        W2.grad = None
        b2.grad = None
    
    if b % 10000 == 0:
        print(b, '/', len(idx), H.item())

0 / 9892431 27.65771484375
10000 / 9892431 19.772968292236328
20000 / 9892431 18.473562240600586
30000 / 9892431 16.911128997802734
40000 / 9892431 15.810965538024902
50000 / 9892431 15.659520149230957
60000 / 9892431 15.101822853088379
70000 / 9892431 14.403911590576172
80000 / 9892431 14.484269142150879
90000 / 9892431 13.810616493225098
100000 / 9892431 13.293817520141602
110000 / 9892431 13.534707069396973
120000 / 9892431 12.978638648986816
130000 / 9892431 12.796838760375977
140000 / 9892431 12.335331916809082
150000 / 9892431 12.296268463134766
160000 / 9892431 12.077373504638672
170000 / 9892431 11.492776870727539
180000 / 9892431 11.54820442199707
190000 / 9892431 11.548019409179688
200000 / 9892431 11.635299682617188
210000 / 9892431 11.018109321594238
220000 / 9892431 10.785733222961426
230000 / 9892431 10.739595413208008
240000 / 9892431 10.452831268310547
250000 / 9892431 10.983940124511719
260000 / 9892431 10.27517032623291
270000 / 9892431 10.196242332458496
280000 / 989

In [16]:
H = 0.
with torch.no_grad():
    for b in range(0,len(testtarget),batchSize):
        e = min(b+batchSize,len(testtarget))
        S = e-b
        batchTarget = torch.tensor(testtarget[b:e], dtype=torch.long, device = device)
        batchContext = testcontext[b:e]
    
        X = E[batchContext].view(S,(order-1) * emb_size)
        h = sigmoid_fn(torch.matmul(X,W1) + b1)
        z = torch.matmul(h,W2) + b2
        H += S * loss_fn(z,batchTarget)

    H /= len(testtarget)

print("Perplexity: ",math.exp(H))

Perplexity:  242.39430915226487
